In [65]:
# @title smulti-threaded sample analysis system

import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from brian2 import *
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pickle
from concurrent.futures import ProcessPoolExecutor
from sklearn.utils import shuffle
import random

try:
    import ipywidgets as widgets
except ImportError:
    !pip3 install ipywidgets
    import ipywidgets as widgets

from IPython.display import display, clear_output

fixed_timesteps = 1001

def get_length(file_path):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    return mfccs.shape[1]

def determine_fixed_length(directory):
    file_paths = []

    for subdir in ['1_4', '2_4', '3_4', '4_4']:
        for file in tqdm(os.listdir(os.path.join(directory, subdir))):
            file_path = os.path.join(directory, subdir, file)
            file_paths.append(file_path)

    # Utilize multiprocessing for faster computation
    with ProcessPoolExecutor() as executor:
        lengths = list(executor.map(get_length, file_paths))

    return min(lengths)

def parallel_data_loader(directories):
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(parallel_load_and_preprocess, directories), total=len(directories)))
    return results

def load_and_preprocess_data_subdir(args):
    directory, subdir = args
    data = []
    labels = []
    
    for file in os.listdir(os.path.join(directory, subdir)):
        file_path = os.path.join(directory, subdir, file)
        processed_data = load_audio(file_path)
        data.append(processed_data)
        label = ['1_4', '2_4', '3_4', '4_4'].index(subdir)
        labels.append(label)
    
    return data, labels

def parallel_load_and_preprocess(directory):
    # Create a pool of processes
    pool = Pool(cpu_count())

    # Create a list of tasks
    tasks = [(directory, time_sig) for time_sig in ['1_4', '2_4', '3_4', '4_4']]

    # Use imap_unordered to distribute the work among the processes
    results = list(tqdm(pool.imap_unordered(load_and_preprocess_data_subdir, tasks), total=len(tasks), mininterval=0.01))

    # Close the pool and wait for all processes to finish
    pool.close()
    pool.join()

    # Combine results
    combined_data = []
    combined_labels = []
    
    for data, labels in results:
        combined_data.extend(data)
        combined_labels.extend(labels)
    
    return combined_data, combined_labels


def adjust_fixed_length(features, timesteps):
    # If the array is 1-dimensional
    if len(features.shape) == 1:
        if features.shape[0] > timesteps:
            return features[:timesteps]
        elif features.shape[0] < timesteps:
            padding = np.zeros(timesteps - features.shape[0])
            return np.hstack((features, padding))
        return features
    # If the array is 2-dimensional
    else:
        # If the time axis of the 2D array is greater than timesteps, crop it.
        if features.shape[1] > timesteps:
            return features[:, :timesteps]
        # If the time axis of the 2D array is less than timesteps, pad it.
        elif features.shape[1] < timesteps:
            padding = np.zeros((features.shape[0], timesteps - features.shape[1]))
            return np.hstack((features, padding))
        return features

# Convert real-valued features to Poisson spike trains
def poisson_spike_encoding(data, duration=10, dt=1*ms):
    # Assuming data is normalized between 0 and 1
    rates = data * (1.0/dt)
    spikes = (np.random.rand(*data.shape) < rates*dt).astype(float)
    return spikes

def temporal_binning(data, bin_size):
    """
    Bins the data into chunks of bin_size and returns the average of each chunk.
    """
    # Split the data into chunks of bin_size
    binned_data = [np.mean(data[i:i+bin_size]) for i in range(0, len(data), bin_size)]
    return np.array(binned_data)

def rate_based_encoding(data, min_freq, max_freq):
    """
    Convert onset strengths to spike frequencies.
    data: The input data (should be normalized to [0, 1])
    min_freq: The minimum spike frequency (corresponds to data value of 0)
    max_freq: The maximum spike frequency (corresponds to data value of 1)
    Returns: Spike frequencies corresponding to input data
    """
    return min_freq + data * (max_freq - min_freq)

def extract_bpm_and_instrument(file_path):
    match = re.search(r"instrument_(\d+)_bpm_(\d+)_duration_(\d+)_noise_([\d.]+)", file_path)
    if match:
        instrument = match.group(1)
        bpm = match.group(2)
        duration = match.group(3)
        noise = match.group(4)
        return instrument, bpm, duration, noise
    return None, None, None, None

def moving_average(data, window_size):
    """Compute moving average"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=22050)  # setting sr ensures all files are resampled to this rate
    return [y, sr, file_path]

# Process the audio file into desired features
# Process the audio file into desired features
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=22050)  # setting sr ensures all files are resampled to this rate
    time_signature = file_path.split('/')[-2].replace('_', '/')
    instrument, bpm = extract_bpm_and_instrument(file_path)

    # Extracting onset strength
    onset_strength = librosa.onset.onset_strength(y=y, sr=sr)
    
    # Extracting tempogram
    tempogram = librosa.feature.tempogram(onset_envelope=onset_strength, sr=sr)
    
    # Extracting tempogram
    tempogram_cropped = librosa.feature.tempogram(onset_envelope=onset_strength[20:], sr=sr)
    
    # Adjust the time axis of each feature to fixed_timesteps
    onset_strength_fixed = adjust_fixed_length(onset_strength, fixed_timesteps)
    tempogram_fixed = adjust_fixed_length(tempogram, fixed_timesteps)

    # Stacking features horizontally
    combined_features = np.vstack(poisson_spike_encoding(onset_strength))
    
    # Normalize to range [0, 1]
    encoded_features = (combined_features - np.min(combined_features)) / (np.max(combined_features) - np.min(combined_features))
    
        # Plotting
    plt.figure(figsize=(12, 14))
    plt.title('audio  with {time_signature} time signature, {bpm} bpm, and instrument {instrument}')

    rows = 6
    # 1. Raw audio
    plt.subplot(rows, 1, 1)
    librosa.display.waveshow(y, sr=sr)
    plt.title('Raw Audio')

    # 2. Onset strength
    plt.subplot(rows, 1, 2)
    plt.plot(onset_strength_fixed)
    plt.title('Onset Strength fixed size')
    
    # 2. Onset strength
    plt.subplot(rows, 1, 3)
    onset_strength_normalized = (onset_strength[20:] - np.min(onset_strength[20:])) / (np.max(onset_strength[20:]) - np.min(onset_strength[20:]))
    plt.plot(onset_strength_normalized)
    plt.title('Onset Strength normalized and cropped')
    
    # Add a plot for averaged onset strength
    plt.subplot(rows, 1, 4)
    averaged_onset = moving_average(onset_strength_normalized, window_size=5)  # using a window size of 10, adjust as needed
    plt.plot(averaged_onset)
    plt.title('Averaged Onset Strength')
    
    # 3. Tempogram
    plt.subplot(rows, 1, 5)
    librosa.display.specshow(tempogram_fixed, sr=sr, x_axis='time', y_axis='tempo')
    plt.title('Tempogram fixed')
    
        # 3. Tempogram
    plt.subplot(rows, 1, 6)
    librosa.display.specshow(tempogram_cropped, sr=sr, x_axis='time', y_axis='tempo')
    plt.title('Tempogram cropped')
    
    
    
    plt.tight_layout()
    plt.savefig(f'output_processing_noise_avg/{time_signature.replace("/", "_")}_BPM{bpm}_noise.png')
    
    return encoded_features[20:]


def count_files(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])

# Current directory
directory = '.'

# Loop through all files in the current directory
for filename in os.listdir(directory):
    # Check if the filename ends with '.png' and contains 'spike_train'
    if filename.endswith('.png') and 'spike_train' in filename:
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Remove the file
        os.remove(filepath)
        print(f"Deleted: {filename}", end='\r')
        

# checking shapes
print("Checking shapes...")
fixed_timesteps = determine_fixed_length('training_data')
print(fixed_timesteps)
fixed_timesteps2 = determine_fixed_length('validation_data')
print(fixed_timesteps2)
fixed_timesteps = max(fixed_timesteps, fixed_timesteps2)


# 1. Load and preprocess data
print("Loading and preprocessing training data...")
directories = ['training_data', 'validation_data']
training_data_results, validation_data_results = parallel_data_loader(directories)

training_data, training_labels = training_data_results
validation_data, validation_labels = validation_data_results
print("\nDone with preprocessing!")


Checking shapes...


100%|██████████| 20/20 [00:00<00:00, 95979.50it/s]


410


100%|██████████| 20/20 [00:00<00:00, 134003.32it/s]


410
Loading and preprocessing training data...


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00,  7.43it/s]

100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


Done with preprocessing!


In [66]:
import scipy.signal
from scipy.signal import savgol_filter


class LIFNeuron:
    def __init__(self, tau_m=20.0, v_rest=0.0, v_threshold=1.0, v_reset=0.0, r_m=1.0, dt=1.0):
        self.tau_m = tau_m
        self.v_rest = v_rest
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.r_m = r_m
        self.dt = dt
        self.v = v_rest

    def step(self, i):
        dv = (-self.v + self.v_rest + self.r_m * i) / self.tau_m * self.dt
        self.v += dv
        spike = 0
        if self.v >= self.v_threshold:
            spike = 1
            self.v = self.v_reset
        return spike
    
def generate_lif_spikes(data, neuron):
    spikes = []
    for i in data:
        spike = neuron.step(i)
        spikes.append(spike)
    return np.array(spikes)

def smooth_using_savgol(data, window_size, polynomial_order=3):
    return savgol_filter(data, window_size, polynomial_order)

# Convert real-valued features to Poisson spike trains
def poisson_spike_encoding(data, duration=10, dt=1*ms):
    # Assuming data is normalized between 0 and 1
    rates = data * (1.0/dt)
    spikes = (np.random.rand(*data.shape) < rates*dt).astype(float)
    return spikes

def low_pass_filter(y, sr, cutoff_freq):
    nyq = 0.5 * sr  # Nyquist frequency
    normal_cutoff = cutoff_freq / nyq
    b, a = scipy.signal.butter(6, normal_cutoff, btype='low', analog=False)
    return scipy.signal.filtfilt(b, a, y)

def high_pass_filter(y, sr, cutoff_freq):
    nyq = 0.5 * sr  # Nyquist frequency
    normal_cutoff = cutoff_freq / nyq
    b, a = scipy.signal.butter(6, normal_cutoff, btype='high', analog=False)
    return scipy.signal.filtfilt(b, a, y)

def normalize_data(data):
    """Normalisiert eine Liste von Werten zwischen 0 und 1."""
    min_val = min(data)
    max_val = max(data)
    return [(val - min_val) / (max_val - min_val) for val in data]

def plot_pixel_spectra_norm(item_no, window_size, tau_m, v_rest, v_threshold, v_reset, r_m, dt, high_pass_cutoff):
    y = training_data[item_no][0]
    sr = training_data[item_no][1]
    file_path = training_data[item_no][2]
    # Extract the MFCCs
    mfccs = librosa.feature.mfcc(y=y[20:], sr=sr)
    # y = low_pass_filter(y, sr, low_pass_cutoff)
    # y = high_pass_filter(y, sr, high_pass_cutoff)
    
    time_signature = file_path.split('/')[-2].replace('_', '/')
    instrument, bpm, duration, noise = extract_bpm_and_instrument(file_path)
    print(f"time signature: {time_signature} BPM:{bpm} Noise:{noise} sampling rate: {sr} ", end='\r')

    # Extracting onset strength
    onset_strength = librosa.onset.onset_strength(y=y, sr=sr)
    
    # Extracting tempogram
    tempogram = librosa.feature.tempogram(onset_envelope=onset_strength, sr=sr)
    
    # Extracting tempogram
    tempogram_cropped = librosa.feature.tempogram(onset_envelope=onset_strength[20:], sr=sr)
    
    # Adjust the time axis of each feature to fixed_timesteps
    onset_strength_fixed = adjust_fixed_length(onset_strength, fixed_timesteps)
    tempogram_fixed = adjust_fixed_length(tempogram, fixed_timesteps)

    # Stacking features horizontally
    combined_features = np.vstack(poisson_spike_encoding(onset_strength))
    
    # Normalize to range [0, 1]
    encoded_features = (combined_features - np.min(combined_features)) / (np.max(combined_features) - np.min(combined_features))
    
    onset_strength_normalized = (onset_strength[20:] - np.min(onset_strength[20:])) / (np.max(onset_strength[20:]) - np.min(onset_strength[20:]))

    averaged_onset = moving_average(onset_strength_normalized, window_size=window_size)  # using a window size of 10, adjust as needed
    normalized_averaged_onset = normalize_data(averaged_onset)
    
    global_tempo = librosa.feature.rhythm.tempo(onset_envelope=onset_strength, sr=sr)[0]
    dtempo = librosa.feature.rhythm.tempo(onset_envelope=onset_strength, sr=sr, aggregate=None)
    
        # Plotting
    plt.figure(figsize=(12, 14))
    plt.title('audio  with {time_signature} time signature, {bpm} bpm, and instrument {instrument}')

    rows = 6
    # 1. Raw audio
    plt.subplot(rows, 1, 1)
    # Prepare time axes for raw audio and onset strength
    time_audio = np.linspace(0, len(y) / sr, len(y))
    hop_length = 512  # default hop length in librosa for onset_strength
    time_onset = np.linspace(0, len(y) / sr, len(normalize_data(onset_strength)))
    # Plot raw audio
    # librosa.display.waveshow(y[20:], sr=sr, alpha=0.7, label='Raw Audio')

    # # Plot onset strength, normalized and scaled to match the raw audio amplitude for visualization
    # plt.plot(onset_strength[20:], color='r', label='Onset Strength')
    

    # Plot raw audio
    plt.plot(time_audio, normalize_data(y), label='Raw Audio', alpha=0.7)

    # Plot onset strength (scaled)
    plt.plot(time_onset, normalize_data(onset_strength), label='Onset Strength (scaled)', alpha=0.7, color='r')

    plt.legend()
    plt.xlabel('Time (s)')

    plt.title(f'Raw Audio and Onset Strength, predicted BPM = {global_tempo}, actual BPM = {bpm}, instrument {instrument} and time signature {time_signature}')
    plt.legend()

    # Frequency spectrum
    plt.subplot(rows, 1, 2)
    # fourier = np.fft.fft(y[20:])
    # n = len(fourier)
    # frequencies = np.fft.fftfreq(n, 1/sr)  # <-- Adjusted this line
    # fft_x = frequencies[:n//2]
    # fft_y = np.abs(fourier)[:n//2]
    # plt.plot(fft_x, fft_y)
    # plt.title('Frequency Spectrum')
    # plt.xlabel('Frequency (Hz)')
    # plt.ylabel('Magnitude')
    librosa.display.specshow(mfccs, x_axis='time')
    plt.title('MFCC')
    plt.xlabel('Time (s)')
    plt.ylabel('MFCC Coefficients')
    
    
    # 3. Frequency spectrum after high-pass filtering
    plt.subplot(rows, 1, 3)
    y_high_pass = high_pass_filter(y[20:], sr, high_pass_cutoff)
    fourier_high_pass = np.fft.fft(y_high_pass)
    n_high_pass = len(fourier_high_pass)
    frequencies_high_pass = np.fft.fftfreq(n_high_pass, 1/sr) 
    plt.plot(frequencies_high_pass[:n_high_pass//2], np.abs(fourier_high_pass)[:n_high_pass//2])
    plt.title(f'Frequency Spectrum after high-pass filter at {int(high_pass_cutoff)} Hz')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')

    plt.tight_layout()  # <-- Makes the layout cleaner
    #plt.legend()
    
    # 2. Onset strength
    # plt.subplot(rows, 1, 2)
    # plt.plot(onset_strength_fixed, label='Onset Strength')
    # poisson_encoded = poisson_spike_encoding(onset_strength_fixed.reshape(1,-1))
    # plt.plot(poisson_encoded[0], label='Poisson Spike Train', linestyle=':')
    # plt.title('Onset Strength fixed size')
    
    # 2. Onset strength
    plt.subplot(rows, 1, 4)
    plt.plot(onset_strength_normalized, label='Onset Strength', alpha=0.7)
    poisson_encoded = poisson_spike_encoding(onset_strength_normalized.reshape(1,-1), dt=dt*ms)
    # Calculate the spike count
    spike_count = np.sum(poisson_encoded)
    plt.plot(poisson_encoded[0], label='Poisson Spike Train', linestyle=':', color='g')
    plt.title(f'Onset Strength fixed size with Poisson Spike Train - Spike Count: {int(spike_count)}')    
    plt.legend()
    # 2. Onset strength
    plt.subplot(rows, 1, 5)
    # plt.plot(onset_strength_normalized)
   
    lif_neuron = LIFNeuron(tau_m=tau_m, v_rest=v_rest, v_threshold=v_threshold, v_reset=v_reset, r_m=r_m, dt=dt)
    lif_spikes = generate_lif_spikes(onset_strength_normalized, lif_neuron)
    spike_count = np.sum(lif_spikes)
        # 3. Onset strength with LIF spike train
    plt.plot(onset_strength_normalized, label='Onset Strength', alpha=0.7)
    plt.plot(lif_spikes, label='LIF Spike Train', linestyle=':', color='r')
    plt.title(f'Onset Strength with LIF Spike Train, Spike Count: {int(spike_count)}')
    plt.legend()
    
    # # Add a plot for averaged onset strength
    # plt.subplot(rows, 1, 4)
    # plt.plot(normalized_averaged_onset)
    # plt.title('Averaged Onset Strength')
    
    # 3. Tempogram
    plt.subplot(rows, 1, 6)
    librosa.display.specshow(tempogram_fixed, sr=sr, x_axis='time', y_axis='tempo')
    plt.title('Tempogram fixed')
    
    #     # 3. Tempogram
    # plt.subplot(rows, 1, 6)
    # librosa.display.specshow(tempogram_cropped, sr=sr, x_axis='time', y_axis='tempo')
    # plt.title('Tempogram cropped')
    
        # lif_neuron = LIFNeuron(tau_m=5, v_rest=0.0, v_threshold=0.7, v_reset=0.0, r_m=1.0, dt=10)

    plt.figtext(0.15, 0.02, f"Item No: {item_no}, Window Size: {window_size}, tau_m: {tau_m}, v_rest: {v_rest}, "
                        f"v_threshold: {v_threshold}, v_reset: {v_reset}, r_m: {r_m}, dt: {dt}, "
                        f"high_pass_cutoff: {high_pass_cutoff}", ha="left", fontsize=10)
    plt.tight_layout()
    plt.savefig('plot_with_params.png')
    plt.show()
    

def interactive_plot_spec_norm(item_no, window_size, tau_m, v_rest, v_threshold, v_reset, r_m, dt, high_pass_cutoff):
    plot_pixel_spectra_norm(item_no, window_size, tau_m, v_rest, v_threshold, v_reset, r_m, dt, high_pass_cutoff)


if widgets is not None:
    widgets.interact(
        interactive_plot_spec_norm,
        item_no=widgets.IntSlider(min=0, max=len(training_data)-1, value=0, step=1, continuous_update=False, description="Item No."),
        window_size=widgets.IntSlider(min=1, max=400, value=10, step=1, continuous_update=False, description="avg window size"),
        tau_m=widgets.FloatSlider(min=1.0, max=100.0, value=5.0, step=0.5, continuous_update=False, description="tau_m"),
        v_rest=widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.1, continuous_update=False, description="v_rest"),
        v_threshold=widgets.FloatSlider(min=0.0, max=1.0, value=0.6, step=0.1, continuous_update=False, description="v_threshold"),
        v_reset=widgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.1, continuous_update=False, description="v_reset"),
        r_m=widgets.FloatSlider(min=0.1, max=1.0, value=1.0, step=0.1, continuous_update=False, description="r_m"),
        dt=widgets.FloatSlider(min=0.1, max=100.0, value=10.0, step=1.0, continuous_update=False, description="dt"),
        high_pass_cutoff=widgets.FloatSlider(min=10.0, max=1000.0, value=500.0, step=10.0, continuous_update=False, description="high_pass_cutoff [Hz]"),
    );

interactive(children=(IntSlider(value=0, continuous_update=False, description='Item No.', max=79), IntSlider(v…